In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from openai import OpenAI
import getpass

api_key = getpass.getpass("Enter your OpenAI API Key:")

client = OpenAI(api_key = api_key)

In [3]:
def get_completion(prompt, model="gpt-4o-mini", temperature = 0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

## Clasificaciones de facturas para el modelo   

In [4]:
import pandas as pd

In [5]:
pip install pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


In [6]:
archivo = './datos.xlsx' 
df = pd.read_excel(archivo)

# Mostrar las primeras filas
print(df.head(5))

   Bill_date  Gross_amount   VAT_amount  Net_amount  VAT_rate %    VAT_Number  \
0 2024-06-25   7205.550000  1250.550000        5955    0.210000     G44970168   
1 2024-05-02   8099.748558  1405.748558        6694    0.210001     B98844574   
2 2024-05-02   8594.000000     0.000000        8594    0.000000     B98844574   
3 2024-08-31   8669.000000     0.000000        8669    0.000000  BE0885184584   
4 2024-06-11   2787.842322   483.842322        2304    0.210001     B82468364   

                              Supplier Bill_Number Quarter  
0  ASOCIACION OPEN DE ARBITRAJE(SPAIN)    157/2024      Q2  
1                 ASPY Prevención, SLU  2024051123      Q2  
2                 ASPY Prevención, SLU  2024051123      Q2  
3               Astrea bv cvba - SPAIN     2471794      Q3  
4                        ATD Auditores           7      Q2  


In [7]:
prompt = f"""
You are an expert in  Spanish Tax, categorize invoices (df) into the ones that should be declared in the model 347.

The list of invoices is provided in df.

Follow this steps and write Python code for each of them:
1. Group the invoices by supplier and calculate the total gross amount for each supplier.
2. Filter out suppliers whose total gross amount is less than 3,005.06 euros.
3. For the remaining suppliers, sum the gross amounts by quarter.
4. Create a final DataFrame with the required information.


Example of df Structure:

Bill date: 25/06/2024
Gross amount:7,205.55
VAT amount:1,250.55
Net amount:5,955.00
VAT rate %:0.21
VAT Number:	G44970168
Supplier:ASOCIACION OPEN DE ARBITRAJE
Bill Number:157/2024
Quarter:Q2


Instructions:

Gross amount: calculate the total gross amount for each supplier.
Supplier: Filter out suppliers whose total gross amount is less than 3,005.06 euros.
Quarter: sum the gross amounts by quarter.

Use information in the df.

df: ```{df}```
"""

response = get_completion(prompt)
print(response)

To categorize the invoices into those that should be declared in the model 347, we can follow the steps outlined in your instructions. Below is the Python code that accomplishes this using the pandas library.

```python
import pandas as pd

# Sample data creation (you would replace this with your actual DataFrame)
data = {
    'Bill_date': ['2024-06-25', '2024-05-02', '2024-05-02', '2024-08-31', '2024-06-11', 
                  '2024-06-19', '2024-05-25', '2024-06-06', '2024-06-06', '2024-01-20', 
                  '2024-05-20', '2024-06-13', '2024-06-13', '2024-02-21', '2024-05-21', 
                  '2024-05-27', '2024-12-27', '2024-06-13', '2024-06-13'],
    'Gross_amount': [7205.55, 8099.75, 8594.00, 8669.00, 2787.84, 
                     1210.00, 9365.40, 12080.44, 4624.00, 11142.75, 
                     6794.00, 5392.68, 9640.00, 6059.67, 9750.00, 
                     8467.60, 9029.00, 10486.50, 8760.00],
    'VAT_amount': [1250.55, 1405.75, 0.00, 0.00, 483.84, 
             

In [ ]:
#Comprobar el código proporcionado:

# Step 1: Group the invoices by supplier and calculate the total gross amount for each supplier
supplier_totals = df.groupby('Supplier')['Gross_amount'].sum().reset_index()

# Step 2: Filter out suppliers whose total gross amount is less than 3,005.06 euros
filtered_suppliers = supplier_totals[supplier_totals['Gross_amount'] >= 3005.06]

# Step 3: For the remaining suppliers, sum the gross amounts by quarter
# First, we need to merge the filtered suppliers back to the original DataFrame to get the quarters
merged_df = df[df['Supplier'].isin(filtered_suppliers['Supplier'])]
quarterly_totals = merged_df.groupby(['Supplier', 'Quarter'])['Gross_amount'].sum().reset_index()

# Step 4: Create a final DataFrame with the required information
final_df = quarterly_totals.rename(columns={'Gross_amount': 'Total_Gross_Amount'})

# Display the final DataFrame
print(final_df)

                              Supplier Quarter  Total_Gross_Amount
0  ASOCIACION OPEN DE ARBITRAJE(SPAIN)      Q2         7205.550000
1                 ASPY Prevención, SLU      Q2        16693.748558
2               Astrea bv cvba - SPAIN      Q3         8669.000000
3            Cheque Dejeuner España SA      Q1        17202.421719
4            Cheque Dejeuner España SA      Q2        85360.625687
5            Cheque Dejeuner España SA      Q4         9029.000000
